# **League of Legends API**

After doing the research I want to model some interesting facts, dimensions and messurements.

facts: Quantitive information.

    . kills
    . deaths
    . farm
    . pings

Dimensions: Description of facts or numbers.

    . Player
    . item
    . champion
   
measurements

    . amount of match won by day
    . most used items in the last month
    . Avg pings per win vs loose match.


All documentation can be found here https://developer.riotgames.com/apis

In [88]:
import os
import requests
import pandas as pd
from time import sleep 
from datetime import datetime
from datetime import timedelta
from dotenv import load_dotenv  

In [89]:
load_dotenv("C:/Users/esposito/workspace/Datos/Ingenieria de Datos/Codin Eric/lol_analyzer/.env")

api_key = os.getenv('API_KEY')
print(api_key)

RGAPI-2d736e6a-2fd2-4884-bfcb-2d5837bcb423


## **Account**

In [90]:
summ_name= 'Faker'
endpoint = f'https://la2.api.riotgames.com/lol/summoner/v4/summoners/by-name/{summ_name}?api_key={api_key}'
try:
    summoner = requests.get(endpoint).json()
except requests.exceptions.ConnectionError as e:
    summoner = "No response"

# summ_name= 'Faker'
# endpoint = f'https://nal.api.riotgames.com/lol/summoner/v4/summoners/by-name/{summ_name}?api_key={api_key}'
# summoner = requests.get(endpoint).json()
# df = pd.DataFrame([summoner])
# df
    


In [91]:
df = pd.DataFrame([summoner])
df

,id,accountId,puuid,name,profileIconId,revisionDate,summonerLevel
0,R8z7R4BSdYWj7a6Hnz3HhtnBiZcYcMgfFZsVBpPzfSp8lg,_nMnFicQ8nJJxsKk5068YJ8sC1SoeS7lCIDwuJ9sCS4z,_Euf38_V0HyPLEZt6GLy9m4C5h4ruRrYH4PYnjRwz8TqgU...,faker,23,1704662058898,568


## **Extract and Concat ranking mastery**

In [92]:
summ_id= summoner['id']
endpoint = f'https://la2.api.riotgames.com/lol/league/v4/entries/by-summoner/{summ_id}?api_key={api_key}'

try:
    res= requests.get(endpoint).json()
except requests.exceptions.ConnectionError as e:
    res = "No response"
    
df_rank = pd.DataFrame([res[0]])
df_rank

,leagueId,queueType,tier,rank,summonerId,summonerName,leaguePoints,wins,losses,veteran,inactive,freshBlood,hotStreak
0,f9dc44e9-015d-4e42-8a04-01204b333246,RANKED_FLEX_SR,PLATINUM,IV,R8z7R4BSdYWj7a6Hnz3HhtnBiZcYcMgfFZsVBpPzfSp8lg,faker,44,48,47,False,False,False,False


In [93]:
df_summ = pd.concat([df,df_rank],axis =1)
df_summ

,id,accountId,puuid,name,profileIconId,revisionDate,summonerLevel,leagueId,queueType,tier,rank,summonerId,summonerName,leaguePoints,wins,losses,veteran,inactive,freshBlood,hotStreak
0,R8z7R4BSdYWj7a6Hnz3HhtnBiZcYcMgfFZsVBpPzfSp8lg,_nMnFicQ8nJJxsKk5068YJ8sC1SoeS7lCIDwuJ9sCS4z,_Euf38_V0HyPLEZt6GLy9m4C5h4ruRrYH4PYnjRwz8TqgU...,faker,23,1704662058898,568,f9dc44e9-015d-4e42-8a04-01204b333246,RANKED_FLEX_SR,PLATINUM,IV,R8z7R4BSdYWj7a6Hnz3HhtnBiZcYcMgfFZsVBpPzfSp8lg,faker,44,48,47,False,False,False,False


In [94]:
df_summ.dtypes

id               object
accountId        object
puuid            object
name             object
profileIconId     int64
revisionDate      int64
summonerLevel     int64
leagueId         object
queueType        object
tier             object
rank             object
summonerId       object
summonerName     object
leaguePoints      int64
wins              int64
losses            int64
veteran            bool
inactive           bool
freshBlood         bool
hotStreak          bool
dtype: object

There is a problem with the concatenate data. Here we have duplicated or unnecesary rows ``` summonerId```, ``` summonerName ```

In [95]:
##drop summonerId and summonerName columns

df_summ = df_summ.drop(columns=['summonerId', 'summonerName'])


In [96]:
df_summ

,id,accountId,puuid,name,profileIconId,revisionDate,summonerLevel,leagueId,queueType,tier,rank,leaguePoints,wins,losses,veteran,inactive,freshBlood,hotStreak
0,R8z7R4BSdYWj7a6Hnz3HhtnBiZcYcMgfFZsVBpPzfSp8lg,_nMnFicQ8nJJxsKk5068YJ8sC1SoeS7lCIDwuJ9sCS4z,_Euf38_V0HyPLEZt6GLy9m4C5h4ruRrYH4PYnjRwz8TqgU...,faker,23,1704662058898,568,f9dc44e9-015d-4e42-8a04-01204b333246,RANKED_FLEX_SR,PLATINUM,IV,44,48,47,False,False,False,False


In [97]:
df['revisionDate']

0    1704662058898
Name: revisionDate, dtype: int64

In [98]:
#Date format is not correct. Let's transform revisionDate from epoch to datetime

df['revisionDate'] = pd.to_datetime(df['revisionDate'], unit ='ms') 
df['revisionDate']

0   2024-01-07 21:14:18.898
Name: revisionDate, dtype: datetime64[ns]

Save the player data into a csv file

In [99]:
df.to_csv(f'data/summoner_{datetime.today().strftime("%Y_m_%d")}.csv',index=False)

## **Champion mastery**

In [100]:
server = 'la2'
puuid_id = summoner['puuid']

endpoint = f'https://{server}.api.riotgames.com/lol/champion-mastery/v4/champion-masteries/by-puuid/{puuid_id}/top?api_key={api_key}'
res = requests.get(endpoint).json()

In [101]:
df_champs = pd.DataFrame(res)
df_champs

,puuid,championId,championLevel,championPoints,lastPlayTime,championPointsSinceLastLevel,championPointsUntilNextLevel,chestGranted,tokensEarned,summonerId
0,_Euf38_V0HyPLEZt6GLy9m4C5h4ruRrYH4PYnjRwz8TqgU...,77,7,374540,1704527982000,352940,0,True,0,xkKJiQ5gDpdinwTzT9w8fjR9wsUtWIhtFtkW8iYJ7lj1cV2f
1,_Euf38_V0HyPLEZt6GLy9m4C5h4ruRrYH4PYnjRwz8TqgU...,92,7,367838,1703007256000,346238,0,False,0,xkKJiQ5gDpdinwTzT9w8fjR9wsUtWIhtFtkW8iYJ7lj1cV2f
2,_Euf38_V0HyPLEZt6GLy9m4C5h4ruRrYH4PYnjRwz8TqgU...,23,7,312736,1704349010000,291136,0,True,0,xkKJiQ5gDpdinwTzT9w8fjR9wsUtWIhtFtkW8iYJ7lj1cV2f


In [102]:
df_champs['lastPlayTime'] = pd.to_datetime (df_champs['lastPlayTime'], unit ='ms') 

df_champs.to_csv(f'data/champs_mastery_{datetime.today().strftime("%Y_m_%d")}.csv',index=False)

## **Match history**

In [103]:
puuid =  summoner['puuid'] 
match_server='americas.api.riotgames.com'
endpoint = f'https://{match_server}/lol/match/v5/matches/by-puuid/{puuid}/ids?api_key={api_key} &start=0&count=20'

res = requests.get(endpoint).json()

In [104]:
res

['LA2_1374110177',
 'LA2_1374097098',
 'LA2_1373967627',
 'LA2_1373957683',
 'LA2_1373932163',
 'LA2_1373925241',
 'LA2_1373678033',
 'LA2_1373672290',
 'LA2_1373661222',
 'LA2_1373651272',
 'LA2_1373630172',
 'LA2_1373365757',
 'LA2_1373351937',
 'LA2_1373344212',
 'LA2_1373341224',
 'LA2_1373329794',
 'LA2_1373320948',
 'LA2_1373060130',
 'LA2_1373052960',
 'LA2_1373048942']

In [105]:
match_id = res[0]
endpoint  = f'https://{match_server}/lol/match/v5/matches/{match_id}?api_key={api_key}'

res = requests.get(endpoint).json()
res

{'metadata': {'dataVersion': '2',
  'matchId': 'LA2_1374110177',
  'participants': ['3HKVimd_Ay9zElV_ujkBChD3NLBEegDBLfGwAKqh9VDd68jklk578vq566ySMxJnT3Pvm6SWNG6few',
   'IvBsXf8Jjtw_WvpsdUIRyrxkzKK8FCsL-TD_aIxDz_xm7zQcxB9OPp5xNcY_DyOCKMFbILJBSND4ew',
   'kS6R05XxQoYz67KGDqUwjolWMMuRqU4UtJ7Rkf29busjratBe5k_xdfJ1dHUe2Rmk5R6SMNsBCaOZA',
   'LM5Nj5kzqE9GTmn6F6_h049umaMxnGEbHL7eK5A68lfLafKAalPWPQ4EJH4IzjnF_wGwtLP4PXNiQQ',
   'FixnFpXKqrHz2fR2_1z98sCWLSP1-7HVKLv--rQyjfOFotNXrswCh9grbjcl_fUmKpu0j7nSLjzNYA',
   'PYriqNSc9b9snWwEHa12C2zQgK27wI5_9yZ2RBLjsJdjqru_O3qzkDZAt3_eHNcUt2Qfh_1ujmn8DQ',
   'Wm_DVSaNLjGnDoLWzmCfmBwWyyNz4h_6efflAqhglHDIhPQm3JVkuXBamwN5JKuzraoA7CYyPc2Phg',
   '_Euf38_V0HyPLEZt6GLy9m4C5h4ruRrYH4PYnjRwz8TqgUlnbGOxxEeIHmTqODNy8ivAviDWNPgSAg',
   'K1Xu2RHvKXpsqeQU4Ig1lznyzl5FHXlc7MzFmQguZ-q7yf46KgudvRFYWNMPnYiC3WEEK8Gq-y_htQ',
   'CBeds1TM9GqRA3kbTgQWrIg4jTv3bI1lWmnlBM5vnrm0xbbEKM2ypd159ZycMfGoqrUH7XlVGQ-Yyw']},
 'info': {'gameCreation': 1704660187046,
  'gameDuration': 1794,
  

In [106]:
basics = [ 
'puuid', 
'win',
'lane',
'role',
'teamPosition',
'individualPosition',
'kills',
'deaths',
'totalMinionsKilled',
'eligibleForProgression',
'timePlayed',
'gameEndedInEarlySurrender', 
'gameEndedInSurrender',
]

kills = [
'killingSprees',
'firstBloodAssist',
'firstBloodKill',
'firstTowerAssist',
'firstTowerKill',
'doubleKills',
'tripleKills',
'quadraKills',
'pentaKills',

]
wards = [
'wardsKilled',
'wardsPlaced', 
'visionWardsBoughtInGame',
'detectorWardsPlaced',
]

objectives = [
'objectivesStolen',
'turretKills',
'dragonKills',
'inhibitorKills', 
'nexusKills',
]

pings = [
'dangerPings',
'enemyMissingPings', 
'enemyVisionPings', 
'getBackPings', 
'holdPings',
'needVisionPings', 
'onMyWayPings', 
'pushPings',
'visionClearedPings',
]

misc = [
'goldEarned',
'goldSpent',
'inhibitorTakedowns',
'inhibitorsLost', 
'itemsPurchased', 
'largestCriticalStrike', 
'largestKillingSpree', 
'largestMultiKill',
'longestTimeSpentLiving', 
'magicDamageDealt',
'magicDamageDealtToChampions', 
'magicDamageTaken', 
'neutralMinionsKilled',
'nexusLost', 
'nexusTakedowns',
'objectivesStolenAssists',
'participantId',
'physicalDamageDealt',
'physicalDamageDealtToChampions',
'physicalDamageTaken',
'sightWardsBoughtInGame',
'spell1Casts',
'spell2Casts',
'spell3Casts',
'spell4Casts',
'summoner1Casts',
'summoner1Id',
'summoner2Casts', 
'summoner2Id',
'teamEarlySurrendered',
'teamId',
'timeCCingOthers',
'totalDamageDealt',
'totalDamageDealtToChampions',
'totalDamageShieldedOnTeammates',
'totalDamageTaken',
'totalHeal',
'totalHealsOnTeammates',
'totalTimeCCDealt',
'totalTimeSpentDead',
'totalUnitsHealed',
'trueDamageDealt',
'trueDamageDealtToChampions',
'trueDamageTaken',
'turretTakedowns',
'turretsLost',
'unrealKills',
'visionScore',
]

stats = basics + kills + wards+ objectives + pings + misc

In [107]:
df = pd.DataFrame(res['info']['participants'])
df = df[stats]
df

,puuid,win,lane,role,teamPosition,individualPosition,kills,deaths,totalMinionsKilled,eligibleForProgression,...,totalTimeCCDealt,totalTimeSpentDead,totalUnitsHealed,trueDamageDealt,trueDamageDealtToChampions,trueDamageTaken,turretTakedowns,turretsLost,unrealKills,visionScore
0,3HKVimd_Ay9zElV_ujkBChD3NLBEegDBLfGwAKqh9VDd68...,False,JUNGLE,NONE,TOP,TOP,5,5,156,True,...,217,206,1,10345,128,1511,1,8,0,18
1,IvBsXf8Jjtw_WvpsdUIRyrxkzKK8FCsL-TD_aIxDz_xm7z...,False,JUNGLE,NONE,JUNGLE,JUNGLE,3,9,18,True,...,362,281,1,25537,964,1314,0,8,0,16
2,kS6R05XxQoYz67KGDqUwjolWMMuRqU4UtJ7Rkf29busjra...,False,MIDDLE,SOLO,MIDDLE,MIDDLE,4,5,151,True,...,161,171,1,9109,1356,232,0,8,0,14
3,LM5Nj5kzqE9GTmn6F6_h049umaMxnGEbHL7eK5A68lfLaf...,False,BOTTOM,CARRY,BOTTOM,BOTTOM,8,6,189,True,...,59,194,1,21036,170,334,1,8,0,26
4,FixnFpXKqrHz2fR2_1z98sCWLSP1-7HVKLv--rQyjfOFot...,False,BOTTOM,SUPPORT,UTILITY,UTILITY,1,9,33,True,...,240,273,5,0,0,865,2,8,0,36
5,PYriqNSc9b9snWwEHa12C2zQgK27wI5_9yZ2RBLjsJdjqr...,True,TOP,SOLO,TOP,TOP,5,2,164,True,...,97,81,1,23154,2832,1114,2,3,0,11
6,Wm_DVSaNLjGnDoLWzmCfmBwWyyNz4h_6efflAqhglHDIhP...,True,JUNGLE,NONE,JUNGLE,JUNGLE,12,2,32,True,...,558,16,1,44052,1110,502,3,3,0,23
7,_Euf38_V0HyPLEZt6GLy9m4C5h4ruRrYH4PYnjRwz8TqgU...,True,MIDDLE,SOLO,MIDDLE,MIDDLE,9,6,186,True,...,128,237,1,0,0,522,3,3,0,12
8,K1Xu2RHvKXpsqeQU4Ig1lznyzl5FHXlc7MzFmQguZ-q7yf...,True,BOTTOM,CARRY,BOTTOM,BOTTOM,6,3,183,True,...,109,93,3,8150,315,0,2,3,0,16
9,CBeds1TM9GqRA3kbTgQWrIg4jTv3bI1lWmnlBM5vnrm0xb...,True,BOTTOM,SUPPORT,UTILITY,UTILITY,1,8,49,True,...,263,207,1,2752,0,479,2,3,0,58


In [108]:
df.to_csv(f'data/match_stats_{datetime.today().strftime("%Y%m%d_%H%M")}.csv',index=False)

In [109]:
version_lst = requests.get('https://ddragon.leagueoflegends.com/api/versions.json').json()
res = requests.get(f'https://ddragon.leagueoflegends.com/cdn/{version_lst[0]}/data/en_US/item.json').json()
res

{'type': 'item',
 'version': '13.24.1',
 'basic': {'name': '',
  'rune': {'isrune': False, 'tier': 1, 'type': 'red'},
  'gold': {'base': 0, 'total': 0, 'sell': 0, 'purchasable': False},
  'group': '',
  'description': '',
  'colloq': ';',
  'plaintext': '',
  'consumed': False,
  'stacks': 1,
  'depth': 1,
  'consumeOnFull': False,
  'from': [],
  'into': [],
  'specialRecipe': 0,
  'inStore': True,
  'hideFromAll': False,
  'requiredChampion': '',
  'requiredAlly': '',
  'stats': {'FlatHPPoolMod': 0,
   'rFlatHPModPerLevel': 0,
   'FlatMPPoolMod': 0,
   'rFlatMPModPerLevel': 0,
   'PercentHPPoolMod': 0,
   'PercentMPPoolMod': 0,
   'FlatHPRegenMod': 0,
   'rFlatHPRegenModPerLevel': 0,
   'PercentHPRegenMod': 0,
   'FlatMPRegenMod': 0,
   'rFlatMPRegenModPerLevel': 0,
   'PercentMPRegenMod': 0,
   'FlatArmorMod': 0,
   'rFlatArmorModPerLevel': 0,
   'PercentArmorMod': 0,
   'rFlatArmorPenetrationMod': 0,
   'rFlatArmorPenetrationModPerLevel': 0,
   'rPercentArmorPenetrationMod': 0,
   

In [110]:
item_list = []

for iid, data in res['data'].items():
    item = {
        'id': iid,
        'name': data['name'],
        'description': data['description'],
        'plaintext':data['plaintext'],
        'gold_base':data['gold']['base'], 
        'gold sell':data['gold']['sell'],
}
    item_list.append(item)
    
pd.DataFrame(item_list)

,id,name,description,plaintext,gold_base,gold sell
0,1001,Boots,<mainText><stats><attention>25</attention> Mov...,Slightly increases Move Speed,300,210
1,1004,Faerie Charm,<mainText><stats><attention>50%</attention> Ba...,Slightly increases Mana Regen,250,175
2,1006,Rejuvenation Bead,<mainText><stats><attention>100%</attention> B...,Slightly increases Health Regen,300,120
3,1011,Giant's Belt,<mainText><stats><attention>350</attention> He...,Greatly increases Health,500,630
4,1018,Cloak of Agility,<mainText><stats><attention>15%</attention> Cr...,Increases critical strike chance,600,420
...,...,...,...,...,...,...
449,7032,Flicker,<mainText><stats><ornnBonus>85</ornnBonus> Att...,,0,2380
450,7033,Cry of the Shrieking City,<mainText><stats><ornnBonus>60</ornnBonus> Abi...,,0,1610
451,7050,Gangplank Placeholder,,New Gangplank interface coming soon!,0,0
452,8001,Anathema's Chains,<mainText><stats><attention>650</attention> He...,,800,1750


In [111]:
version_lst = requests.get('https://ddragon.leagueoflegends.com/api/versions.json').json()
res = requests.get(f'https://ddragon.leagueoflegends.com/cdn/{version_lst[0]}/data/en_US/champion.json').json()
res

{'type': 'champion',
 'format': 'standAloneComplex',
 'version': '13.24.1',
 'data': {'Aatrox': {'version': '13.24.1',
   'id': 'Aatrox',
   'key': '266',
   'name': 'Aatrox',
   'title': 'the Darkin Blade',
   'blurb': 'Once honored defenders of Shurima against the Void, Aatrox and his brethren would eventually become an even greater threat to Runeterra, and were defeated only by cunning mortal sorcery. But after centuries of imprisonment, Aatrox was the first to find...',
   'info': {'attack': 8, 'defense': 4, 'magic': 3, 'difficulty': 4},
   'image': {'full': 'Aatrox.png',
    'sprite': 'champion0.png',
    'group': 'champion',
    'x': 0,
    'y': 0,
    'w': 48,
    'h': 48},
   'tags': ['Fighter', 'Tank'],
   'partype': 'Blood Well',
   'stats': {'hp': 650,
    'hpperlevel': 114,
    'mp': 0,
    'mpperlevel': 0,
    'movespeed': 345,
    'armor': 38,
    'armorperlevel': 4.45,
    'spellblock': 32,
    'spellblockperlevel': 2.05,
    'attackrange': 175,
    'hpregen': 3,
    'hp

In [112]:
champ_list = []

for iid, data in res['data'].items():
    champ = {
        'id': iid,
        'name': data['name'], 
        'title': data['title'],
        'blurb': data['blurb'], 
        'tags' : data['tags'], 
        'partype': data['partype'], 
        'stats': data['stats'],
}
    champ_list.append(champ)

pd.DataFrame (champ_list)

,id,name,title,blurb,tags,partype,stats
0,Aatrox,Aatrox,the Darkin Blade,Once honored defenders of Shurima against the ...,"[Fighter, Tank]",Blood Well,"{'hp': 650, 'hpperlevel': 114, 'mp': 0, 'mpper..."
1,Ahri,Ahri,the Nine-Tailed Fox,Innately connected to the magic of the spirit ...,"[Mage, Assassin]",Mana,"{'hp': 590, 'hpperlevel': 96, 'mp': 418, 'mppe..."
2,Akali,Akali,the Rogue Assassin,Abandoning the Kinkou Order and her title of t...,[Assassin],Energy,"{'hp': 570, 'hpperlevel': 119, 'mp': 200, 'mpp..."
3,Akshan,Akshan,the Rogue Sentinel,"Raising an eyebrow in the face of danger, Aksh...","[Marksman, Assassin]",Mana,"{'hp': 630, 'hpperlevel': 107, 'mp': 350, 'mpp..."
4,Alistar,Alistar,the Minotaur,Always a mighty warrior with a fearsome reputa...,"[Tank, Support]",Mana,"{'hp': 685, 'hpperlevel': 120, 'mp': 350, 'mpp..."
...,...,...,...,...,...,...,...
161,Zeri,Zeri,The Spark of Zaun,"A headstrong, spirited young woman from Zaun's...",[Marksman],Mana,"{'hp': 630, 'hpperlevel': 110, 'mp': 250, 'mpp..."
162,Ziggs,Ziggs,the Hexplosives Expert,"With a love of big bombs and short fuses, the ...",[Mage],Mana,"{'hp': 606, 'hpperlevel': 106, 'mp': 480, 'mpp..."
163,Zilean,Zilean,the Chronokeeper,"Once a powerful Icathian mage, Zilean became o...","[Support, Mage]",Mana,"{'hp': 574, 'hpperlevel': 96, 'mp': 452, 'mppe..."
164,Zoe,Zoe,the Aspect of Twilight,"As the embodiment of mischief, imagination, an...","[Mage, Support]",Mana,"{'hp': 630, 'hpperlevel': 106, 'mp': 425, 'mpp..."


In [113]:
# include data into dataframe

df_data = pd.DataFrame([data]) 
df_data

,version,id,key,name,title,blurb,info,image,tags,partype,stats
0,13.24.1,Zyra,143,Zyra,Rise of the Thorns,"Born in an ancient, sorcerous catastrophe, Zyr...","{'attack': 4, 'defense': 3, 'magic': 8, 'diffi...","{'full': 'Zyra.png', 'sprite': 'champion5.png'...","[Mage, Support]",Mana,"{'hp': 574, 'hpperlevel': 93, 'mp': 418, 'mppe..."


In [114]:
# using json normalized 

df_stats = pd.json_normalize(df_data['stats']) 
df_stats 

,hp,hpperlevel,mp,mpperlevel,movespeed,armor,armorperlevel,spellblock,spellblockperlevel,attackrange,hpregen,hpregenperlevel,mpregen,mpregenperlevel,crit,critperlevel,attackdamage,attackdamageperlevel,attackspeedperlevel,attackspeed
0,574,93,418,25,340,29,4.2,30,1.3,575,5.5,0.5,13,0.4,0,0,53,3.2,2.11,0.681


In [115]:
pd.concat([df_data, df_stats],axis=1)

,version,id,key,name,title,blurb,info,image,tags,partype,...,hpregen,hpregenperlevel,mpregen,mpregenperlevel,crit,critperlevel,attackdamage,attackdamageperlevel,attackspeedperlevel,attackspeed
0,13.24.1,Zyra,143,Zyra,Rise of the Thorns,"Born in an ancient, sorcerous catastrophe, Zyr...","{'attack': 4, 'defense': 3, 'magic': 8, 'diffi...","{'full': 'Zyra.png', 'sprite': 'champion5.png'...","[Mage, Support]",Mana,...,5.5,0.5,13,0.4,0,0,53,3.2,2.11,0.681
